- https://arxiv.org/abs/1505.04597
    - U-Net: Convolutional Networks for Biomedical Image Segmentation


In [1]:
from IPython.display import Image

## architecture

In [7]:
# https://arxiv.org/abs/1505.04597
Image(url='https://huggingface.co/blog/assets/78_annotated-diffusion/unet_architecture.jpg', width=800)

In [5]:
Image(url='https://camo.githubusercontent.com/2c676413dda1f487521dd5c1e5c4b35b8cfbf06d50880e15660ea44bd76eac6f/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f756e65742d6d6f64656c2e706e67', 
              width=600)

## UNet => DiT